In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import root_scalar

In [3]:
# csv_files = ["/data/kebl6672/dpo-toxic-general/toxicity/gemma2_dpo_neuron_cossims.csv", "/data/kebl6672/dpo-toxic-general/toxicity/gemma2_dpo_neuron_projections.csv", "/data/kebl6672/dpo-toxic-general/toxicity/gemma2_neuron_cossims.csv", "/data/kebl6672/dpo-toxic-general/toxicity/gemma2_neuron_projections.csv"]

# # Read all CSV files into dataframes
# dfs = [pd.read_csv(file) for file in csv_files]

# # Merge dataframes on 'layer_idx' and 'neuron_idx'
# merged_df = dfs[0]
# for df in dfs[1:]:
#     merged_df = pd.merge(merged_df, df, on=['layer_idx', 'neuron_idx'], how='outer')

# # Save the merged dataframe to a new CSV file
# merged_df.to_csv("/data/kebl6672/dpo-toxic-general/toxicity/gemma2_all_neuron_metrics.csv", index=False)

# # Display the first few rows
# print(merged_df.head())

   layer_idx  neuron_idx  dpo_cosine_similarity  dpo_projection_value  \
0          0           0               0.000140              0.000001   
1          0           1               0.006405             -0.000042   
2          0           2               0.002937              0.000007   
3          0           3              -0.000340              0.000001   
4          0           4              -0.048767              0.000382   

   dpo_activation_value  pt_cosine_similarity  pt_projection_value  \
0              0.023575             -0.000066        -5.960464e-07   
1             -0.016129              0.006180        -2.896786e-05   
2              0.006214              0.002983         5.602837e-06   
3             -0.007721             -0.000586         4.172325e-07   
4             -0.022491             -0.048981         3.707409e-04   

   pt_activation_value  
0             0.024582  
1            -0.011658  
2             0.004894  
3            -0.001814  
4            -0

In [8]:
csv_files = ["/data/kebl6672/dpo-toxic-general/toxicity/mistral_neuron_cossims.csv", "/data/kebl6672/dpo-toxic-general/toxicity/mistral_neuron_projections.csv"]

# Read all CSV files into dataframes
dfs = [pd.read_csv(file) for file in csv_files]

# Merge dataframes on 'layer_idx' and 'neuron_idx'
merged_df = dfs[0]
for df in dfs[1:]:
    merged_df = pd.merge(merged_df, df, on=['layer_idx', 'neuron_idx'], how='outer')

# Save the merged dataframe to a new CSV file
merged_df.to_csv("/data/kebl6672/dpo-toxic-general/toxicity/mistral_embed_neuron_metrics.csv", index=False)

# Display the first few rows
print(merged_df.head())

FileNotFoundError: [Errno 2] No such file or directory: '/data/kebl6672/dpo-toxic-general/toxicity/mistral_neuron_cossims.csv'

In [9]:
# Use old version!
df = pd.read_csv('/data/kebl6672/dpo-toxic-general/toxicity/mistral_embed_neuron_metrics.csv')

In [2]:
# Use old version!
df = pd.read_csv('/data/kebl6672/dpo-toxic-general/toxicity/llama3_all_neuron_metrics.csv')

In [3]:
df = df.replace([np.inf, -np.inf], np.nan).dropna()

In [5]:
# df['activation_diff'] = df['pt_activation_value'] - df['dpo_activation_value']
# df['projection_diff'] = df['pt_projection_value'] - df['dpo_projection_value']

In [4]:
df.head()

,layer_idx,neuron_idx,dpo_cosine_similarity,dpo_projection_value,dpo_activation_value,pt_cosine_similarity,pt_projection_value,pt_activation_value
0,0,0,0.008034,-0.002918,-0.000184,0.008034,-0.002989,-0.000189
1,0,1,-0.000513,-0.000097,0.000095,-0.000513,-0.000133,0.000130
2,0,2,0.002647,-0.001558,-0.000323,0.002647,-0.001447,-0.000300
3,0,3,-0.044128,0.002867,-0.000034,-0.044128,0.010277,-0.000121
4,0,4,0.021164,-0.008995,-0.000219,0.021164,-0.007141,-0.000174


##### Extract all neuron indexes from each neuron group

In [20]:
# Patch all uparrow ones
tuples_list = [
    (int(row['layer_idx']), int(row['neuron_idx']), row['dpo_activation_value'])
    for _, row in df.iterrows()
    if (row['projection_diff'] < 0) 
]

print(len(tuples_list))

220293


In [35]:
df_subset = pd.DataFrame(tuples_list, columns=["layer_idx", "neuron_idx", "assigned_value"])
df_subset.to_csv("mistral_patch_four_neuron_configs.csv", index=False)

In [22]:
# Patch AP
tuples_list = [
    (int(row['layer_idx']), int(row['neuron_idx']), row['dpo_activation_value'])
    for _, row in df.iterrows()
    if (row['pt_cosine_similarity'] < 0 and row['pt_activation_value'] > 0 and row['projection_diff'] > 0) # AP
]

print(len(tuples_list))

48306


In [24]:
# Patch AN
tuples_list = [
    (int(row['layer_idx']), int(row['neuron_idx']), row['dpo_activation_value'])
    for _, row in df.iterrows()
    if (row['pt_cosine_similarity'] < 0 and row['pt_activation_value'] < 0 and row['projection_diff'] > 0) # AN
]

print(len(tuples_list))

70928


In [26]:
# Patch TN
tuples_list = [
    (int(row['layer_idx']), int(row['neuron_idx']), row['dpo_activation_value'])
    for _, row in df.iterrows()
    if (row['pt_cosine_similarity'] > 0 and row['pt_activation_value'] < 0 and row['projection_diff'] > 0) # TN  
]

print(len(tuples_list))

48371


In [28]:
# Patch TP
tuples_list = [
    (int(row['layer_idx']), int(row['neuron_idx']), row['dpo_activation_value'])
    for _, row in df.iterrows()
    if (row['pt_cosine_similarity'] > 0 and row['pt_activation_value'] > 0 and row['projection_diff'] > 0) # TP    
]

print(len(tuples_list))

70830


In [30]:
# Patch TP + AN
tuples_list = [
    (int(row['layer_idx']), int(row['neuron_idx']), row['dpo_activation_value'])
    for _, row in df.iterrows()
    if (row['pt_cosine_similarity'] < 0 and row['pt_activation_value'] < 0 and row['projection_diff'] > 0) or # AN
       (row['pt_cosine_similarity'] > 0 and row['pt_activation_value'] > 0 and row['projection_diff'] > 0) # TP    
]

print(len(tuples_list))

141758


In [32]:
# Patch TP + AN + TN
tuples_list = [
    (int(row['layer_idx']), int(row['neuron_idx']), row['dpo_activation_value'])
    for _, row in df.iterrows()
    if (row['pt_cosine_similarity'] < 0 and row['pt_activation_value'] < 0 and row['projection_diff'] > 0) or # AN
       (row['pt_cosine_similarity'] > 0 and row['pt_activation_value'] < 0 and row['projection_diff'] > 0) or # TN
       (row['pt_cosine_similarity'] > 0 and row['pt_activation_value'] > 0 and row['projection_diff'] > 0) # TP    
]

print(len(tuples_list))

190129


In [34]:
# Patch all four groups
tuples_list = [
    (int(row['layer_idx']), int(row['neuron_idx']), row['dpo_activation_value'])
    for _, row in df.iterrows()
    if (row['pt_cosine_similarity'] < 0 and row['pt_activation_value'] > 0 and row['projection_diff'] > 0) or # AP
       (row['pt_cosine_similarity'] < 0 and row['pt_activation_value'] < 0 and row['projection_diff'] > 0) or # AN
       (row['pt_cosine_similarity'] > 0 and row['pt_activation_value'] < 0 and row['projection_diff'] > 0) or # TN
       (row['pt_cosine_similarity'] > 0 and row['pt_activation_value'] > 0 and row['projection_diff'] > 0) # TP    
]

print(len(tuples_list))


238435


##### Tuning-free group identification

In [12]:
# Halve TP 
tuples_list = [
    (int(row['layer_idx']), int(row['neuron_idx']), row['pt_activation_value']/2)
    for _, row in df.iterrows()
    if (row['pt_cosine_similarity'] > 0 and row['pt_activation_value'] > 0) # TP    
]
print(len(tuples_list))

114673


In [8]:
df_subset = pd.DataFrame(tuples_list, columns=["layer_idx", "neuron_idx", "assigned_value"])
df_subset.to_csv("llama3_1.5_two_free_neuron_configs.csv", index=False)

In [14]:
# Halve AN
tuples_list = [
    (int(row['layer_idx']), int(row['neuron_idx']), row['pt_activation_value']/2)
    for _, row in df.iterrows()
    if (row['pt_cosine_similarity'] < 0 and row['pt_activation_value'] < 0) # AN  
]
print(len(tuples_list))

115029


In [16]:
# Halve TP + AN
tuples_list = [
    (int(row['layer_idx']), int(row['neuron_idx']), row['pt_activation_value']/2)
    for _, row in df.iterrows()
    if (row['pt_cosine_similarity'] < 0 and row['pt_activation_value'] < 0) or # AN
       (row['pt_cosine_similarity'] > 0 and row['pt_activation_value'] > 0) # TP    
]
print(len(tuples_list))

229702


In [18]:
# 1.5* AP 
tuples_list = [
    (int(row['layer_idx']), int(row['neuron_idx']), row['pt_activation_value']*1.5)
    for _, row in df.iterrows()
    if (row['pt_cosine_similarity'] < 0 and row['pt_activation_value'] > 0) # AP
]
print(len(tuples_list))

114557


In [20]:
# 1.5* TN
tuples_list = [
    (int(row['layer_idx']), int(row['neuron_idx']), row['pt_activation_value']*1.5)
    for _, row in df.iterrows()
    if (row['pt_cosine_similarity'] > 0 and row['pt_activation_value'] < 0) # TN  
]
print(len(tuples_list))

114473


In [7]:
# 1.5* (AP + TN)
tuples_list = [
    (int(row['layer_idx']), int(row['neuron_idx']), row['pt_activation_value']*1.5)
    for _, row in df.iterrows()
    if (row['pt_cosine_similarity'] < 0 and row['pt_activation_value'] > 0) or # AP
       (row['pt_cosine_similarity'] > 0 and row['pt_activation_value'] < 0) # TN  
]
print(len(tuples_list))

225815


In [48]:
# 0.75# for TP and AN, 1.25* for AP and TN
tuples_list = [
    (int(row['layer_idx']), int(row['neuron_idx']), row['pt_activation_value'] * 0.75) 
    if (row['pt_cosine_similarity'] > 0 and row['pt_activation_value'] > 0) or  # TP
       (row['pt_cosine_similarity'] < 0 and row['pt_activation_value'] < 0)    # AN
    else (int(row['layer_idx']), int(row['neuron_idx']), row['pt_activation_value'] * 1.25)  
    for _, row in df.iterrows()
    if (row['pt_cosine_similarity'] < 0 and row['pt_activation_value'] > 0) or  # AP
       (row['pt_cosine_similarity'] > 0 and row['pt_activation_value'] < 0) or  # TN
       (row['pt_cosine_similarity'] > 0 and row['pt_activation_value'] > 0) or  # TP
       (row['pt_cosine_similarity'] < 0 and row['pt_activation_value'] < 0)    # AN
]

print(len(tuples_list))

458728


In [24]:
# Halve for TP and AN, 1.5* for AP and TN
tuples_list = [
    (int(row['layer_idx']), int(row['neuron_idx']), row['pt_activation_value'] / 2)  # Halve activation
    if (row['pt_cosine_similarity'] > 0 and row['pt_activation_value'] > 0) or  # TP
       (row['pt_cosine_similarity'] < 0 and row['pt_activation_value'] < 0)    # AN
    else (int(row['layer_idx']), int(row['neuron_idx']), row['pt_activation_value'] * 1.5)  # 1.5*activation
    for _, row in df.iterrows()
    if (row['pt_cosine_similarity'] < 0 and row['pt_activation_value'] > 0) or  # AP
       (row['pt_cosine_similarity'] > 0 and row['pt_activation_value'] < 0) or  # TN
       (row['pt_cosine_similarity'] > 0 and row['pt_activation_value'] > 0) or  # TP
       (row['pt_cosine_similarity'] < 0 and row['pt_activation_value'] < 0)    # AN
]

print(len(tuples_list))


458732
